In [1]:
import os
import datetime
import math
import time
import multiprocessing as mp
from get_today_schedule import get_today_sched
import numpy as np
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
from datetime import date, datetime


'''
This function will run the simulations on a test data set to see how they
perform on last seasons data.
'''

#pull in the entire schedule dataset
engine = create_engine(os.environ.get('DEV_DB_CONNECT'))

sql_query = 'SELECT * from nhl_tables.nhl_schedule'
df = pd.read_sql(sql_query, con=engine,
                 parse_dates = {'game_date': '%Y-%m-%d'})

/Users/MattBarlowe/.virtualenvs/databasescripts/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
season_start = '2017-10-04'
season_end = '2018-04-08'
startdate = datetime.strptime(season_start, '%Y-%m-%d')
enddate = datetime.strptime(season_end, '%Y-%m-%d')
date_list = [date.fromordinal(i) for i in range(startdate.toordinal(), enddate.toordinal()+1)]

In [3]:
type(date_list)

list

In [4]:
def get_avg_df(df):
    reg = df[df.ot_flag != 1]
    ot = df[(df.ot_flag == 1) & (df.shootout_flag != 1)]
    shootout = df[df.shootout_flag == 1]

    reg_avg = reg[['home_score', 'away_score','seconds_in_ot']].mean()
    ot_avg = ot[['home_score', 'away_score', 'seconds_in_ot']].mean()
    shootout_avg = shootout[['home_score', 'away_score', 'seconds_in_ot']].mean()

    reg_avg['ot_flag'] = 0
    ot_avg['ot_flag'] = 1
    shootout_avg['ot_flag'] = 1

    reg_avg['shootout_flag'] = 0
    ot_avg['shootout_flag'] = 0
    shootout_avg['shootout_flag'] = 1

    avg_df = pd.concat([reg_avg, ot_avg, shootout_avg], axis=1).T

    avg_df.columns = ['goals_for', 'goals_against', 'seconds_in_ot', 'ot_flag', 'shootout_flag']
    avg_df['non_ot_goals_for'] = np.where(((avg_df.shootout_flag == 1) | (avg_df.ot_flag == 1)) & 
                                              (avg_df.goals_for > avg_df.goals_against), avg_df.goals_for - 1,
                                              avg_df.goals_for)
    avg_df['non_ot_goals_against'] = np.where(((avg_df.shootout_flag == 1) | (avg_df.ot_flag == 1)) & 
                                              (avg_df.goals_for < avg_df.goals_against), avg_df.goals_against - 1,
                                              avg_df.goals_against)
    avg_df['ot_goals'] = np.where(avg_df.shootout_flag == 0, 
                                          avg_df.goals_for - avg_df.non_ot_goals_for, 0)
    avg_df['ot_goals_against'] = np.where((avg_df.ot_flag == 1) & (avg_df.shootout_flag != 1), 1, 0)
    
    return avg_df

In [5]:
def clean_results(results_df, team, date):
    '''
    this function cleans the results dataframe and just strips out the wanted
    team results and creates a column for OT goals as well 
    '''
    results_df = results_df[results_df.game_date < date]
    cleaned_results = []
    #looping through the results_df to pull out only the games the team variable played in.
    for index, row in results_df.iterrows():
        if row.home_team == team:
            #print(row)
            new_row = row[['game_id', 'game_type', 'season', 'game_date', 'home_team_id', 'home_team',
                           'home_score', 'away_score', 'ot_flag', 'shootout_flag', 'seconds_in_ot']]
            
            #print(new_row)
            new_row.index = ['game_id', 'game_type', 'season', 'game_date', 'team_id', 'team',
                           'goals_for', 'goals_against', 'ot_flag', 'shootout_flag', 'seconds_in_ot']
            
            new_row['is_home'] = 1
            
            cleaned_results.append(new_row)
            
        elif row.away_team == team:
            #print(row)
            new_row = row[['game_id', 'game_type', 'season', 'game_date', 'away_team_id', 'away_team',
                           'away_score', 'home_score', 'ot_flag', 'shootout_flag', 'seconds_in_ot']]
            #print(new_row)
            new_row.index = ['game_id', 'game_type', 'season', 'game_date', 'team_id', 'team',
                           'goals_for', 'goals_against', 'ot_flag', 'shootout_flag', 'seconds_in_ot']
            
            new_row['is_home'] = 0
            
            #print(new_row)
            cleaned_results.append(new_row)
            
    cleaned_df = pd.concat(cleaned_results, axis=1).T
    
    #calculating non ot goals by seeing if the game went to ot or shootout and if so whether the team won or not.
    #if they did then they score one less goals than their final total if not then they scored their same goals for 
    #amount
    cleaned_df['non_ot_goals_for'] = np.where(((cleaned_df.shootout_flag == 1) | (cleaned_df.ot_flag == 1)) & 
                                          (cleaned_df.goals_for > cleaned_df.goals_against), cleaned_df.goals_for - 1,
                                          cleaned_df.goals_for)
    cleaned_df['non_ot_goals_against'] = np.where(((cleaned_df.shootout_flag == 1) | (cleaned_df.ot_flag == 1)) & 
                                          (cleaned_df.goals_for < cleaned_df.goals_against), cleaned_df.goals_against - 1,
                                          cleaned_df.goals_against)
    cleaned_df['ot_goals'] = np.where(cleaned_df.shootout_flag == 0, 
                                      cleaned_df.goals_for - cleaned_df.non_ot_goals_for, 0)
    cleaned_df['ot_goals_against'] = np.where(cleaned_df.shootout_flag == 0,
                                              cleaned_df.goals_against - cleaned_df.non_ot_goals_against,0)
                                            
    cleaned_df = cleaned_df.reset_index(drop=True)
    
    #only return the last two seasons of games
    cleaned_df = cleaned_df.sort_values(by=['game_date'], ascending=False).iloc[:164, :]
    
    return cleaned_df
            

In [6]:
def monte_carlo_predict(home_results, away_results):
    '''
    taking the results of each team this function runs a monte carlo
    simulation to predict goals scored using the poissson distribution
    '''
    
    results =[]
    
    home_lambda = (home_results['non_ot_goals_for'].mean() + away_results['non_ot_goals_against'].mean()) / 2
    away_lambda = (home_results['non_ot_goals_against'].mean() + away_results['non_ot_goals_for'].mean()) / 2
    
    home_ot_lambda = (((home_results['ot_goals'].sum()/home_results['seconds_in_ot'].sum()) * 300) + \
                      (away_results['ot_goals_against'].sum()/away_results['seconds_in_ot'].sum())*300)/2
    
    away_ot_lambda = (((away_results['ot_goals'].sum()/away_results['seconds_in_ot'].sum()) * 300) + \
                      (home_results['ot_goals_against'].sum()/home_results['seconds_in_ot'].sum())*300)/2
    
    home_ot_win_percent = np.where((home_results.ot_flag == 1) & (home_results.shootout_flag == 0)
                                   & (home_results.goals_for > home_results.goals_against), 1, 0).sum()/home_results[(home_results.ot_flag == 1) & (home_results.shootout_flag == 0)].shape[0]
    away_ot_win_percent = np.where((away_results.ot_flag == 1) & (away_results.shootout_flag == 0)
                                   & (away_results.goals_for > away_results.goals_against), 1, 0).sum()/away_results[(away_results.ot_flag == 1) & (away_results.shootout_flag == 0)].shape[0]
    home_so_win_percent = np.where((home_results.shootout_flag == 1) &(home_results.goals_for > home_results.goals_against), 
                                   1, 0).sum()/home_results[home_results.shootout_flag == 1].shape[0]
    away_so_win_percent = np.where((away_results.shootout_flag == 1) & (away_results.goals_for > away_results.goals_against), 
                                   1, 0).sum()/away_results[away_results.shootout_flag == 1].shape[0]
    if math.isnan(home_ot_win_percent):
        home_ot_win_percent = .5
        
    if math.isnan(away_ot_win_percent):
        away_ot_win_percent = .5
        
    if math.isnan(home_so_win_percent):
        home_so_win_percent = .5
        
    if math.isnan(away_so_win_percent):
        away_so_win_percent = .5
    
    home_reg_goals = np.random.poisson(home_lambda, 10000)
    away_reg_goals = np.random.poisson(away_lambda, 10000)
       
    for home, away in zip(home_reg_goals, away_reg_goals):
        if home > away:
            results.append(1)
        elif away > home:
            results.append(0)
        else:
            prob_of_zero_goals = (math.exp(-home_ot_lambda) * math.exp(-away_ot_lambda))
            
            if np.random.binomial(1, prob_of_zero_goals) == 1:
                try:
                    prob_of_home_so_win = home_so_win_percent/(home_so_win_percent + away_so_win_percent)
                except:
                    prob_of_home_so_win = .5
                results.append(np.random.binomial(1, prob_of_home_so_win))
            else:
                try:
                    prob_of_home_ot_win = home_ot_win_percent/(home_ot_win_percent + away_ot_win_percent)
                except:
                    prob_of_home_ot_win = .5
                results.append(np.random.binomial(1, prob_of_home_ot_win))
                
    home_win_prob = sum(results)/len(results)
                
    return home_win_prob
    
    
    

In [9]:
def multi_proc_monte(home_results, away_results, iter=10000):
    
    pool =  mp.Pool(4)
    pool_list = [pool.apply_async(monte_carlo_predict, args=(home_results, away_results)) for _ in range(iter)]
    
    results = [f.get() for f in pool_list]
    pool.close()
    
    return results
    

In [ ]:
#looping over every date of the 2018 season to pull the games from the dataframe 
#that were played on that date
predictions = []

for game_date in date_list:
    #create a dataframe of the games played
    schedule_df = df[df.game_date == pd.Timestamp(game_date).strftime('%Y-%m-%d')]
    train_df = df[df.game_date < pd.Timestamp(game_date).strftime('%Y-%m-%d')]
    ot_df = train_df[(train_df.ot_flag == 1) & (train_df.shootout_flag != 1)]
    avg_scoring_per_game = ((train_df['home_score'].sum() + train_df['away_score'].sum())/df.shape[0])/60
    avg_scoring_per_ot = (ot_df.shape[0]/ot_df.seconds_in_ot.sum()) * 60
    home_win_pct = df.home_win.sum()/df.shape[0]
    
    #home ice advantage multiplier
    home_mult = 1+((home_win_pct - .5)/.5)
    
    #iterating over the dataframe to simulate one game at a time
    for index, row in schedule_df.iterrows():
        game_id = row.game_id
        
        home_team = row.home_team
        away_team = row.away_team
        #all these try and excepts are because vegas has no past samples
        try:
            home_results = clean_results(df, home_team, game_date)
            
            if home_results.shape[0] < 2:
                home_results = get_avg_df(train_df)
                
        except:
            home_results = get_avg_df(train_df)
        try:
            away_results = clean_results(df, away_team, game_date)
            #vegas bullshit
            if away_results.shape[0] < 2:
                away_results = get_avg_df(train_df)
                
        except:
            away_results = get_avg_df(train_df)
        home_win_probabilities = []
        
        #running the monte carlo simulation a 10,000 times
        print(f'{away_team} vs. {home_team}')
        print(pd.Timestamp(game_date).strftime('%Y-%m-%d'))
        print(row.game_id)
        
        #for x in range(0,10000):
        #win_prob = 
        start_time = time.time()
        home_win_probabilities = multi_proc_monte(home_results, away_results)
        end_time = time.time()
        time_to_run = end_time - start_time
        print(f'Time for simulation to run: {time_to_run}')
        #testing to see how it performs without the home multiplier
        #win_prob = home_mult * win_prob
        #home_win_probabilities.append(win_prob)
    #multiply result from monte carlo function by a home team multiplier
        print(sum(home_win_probabilities)/len(home_win_probabilities))
        final_win_probs = sum(home_win_probabilities)/len(home_win_probabilities)
        predictions.append([game_id, game_date, home_team, away_team, final_win_probs])


/Users/MattBarlowe/.virtualenvs/databasescripts/lib/python3.6/site-packages/ipykernel_launcher.py:6: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.
  


Toronto Maple Leafs vs. Winnipeg Jets
2017-10-04
2017020001
Time for simulation to run: 70.84937906265259
0.5085041300000096
St. Louis Blues vs. Pittsburgh Penguins
2017-10-05
2017020002
Time for simulation to run: 72.60064506530762
0.5233823200000034
Calgary Flames vs. Edmonton Oilers
2017-10-05
2017020003
Time for simulation to run: 76.67505216598511
0.5054932600000046
Philadelphia Flyers vs. San Jose Sharks
2017-10-05
2017020004
Time for simulation to run: 79.18061804771423
0.5400541499999921
Nashville Predators vs. Boston Bruins
2017-10-05
2017020005
Time for simulation to run: 73.5757110118866
0.5143165800000047
Colorado Avalanche vs. New York Rangers
2017-10-05
2017020007
Time for simulation to run: 75.84042310714722
0.5889235100000059
Minnesota Wild vs. Detroit Red Wings
2017-10-05
2017020009
Time for simulation to run: 79.03539609909058
0.45171459000000314
Montréal Canadiens vs. Buffalo Sabres
2017-10-05
2017020006
Time for simulation to run: 77.95424580574036
0.451502240000004

/Users/MattBarlowe/.virtualenvs/databasescripts/lib/python3.6/site-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in long_scalars
/Users/MattBarlowe/.virtualenvs/databasescripts/lib/python3.6/site-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in long_scalars
/Users/MattBarlowe/.virtualenvs/databasescripts/lib/python3.6/site-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in long_scalars
/Users/MattBarlowe/.virtualenvs/databasescripts/lib/python3.6/site-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in long_scalars


Time for simulation to run: 67.25498700141907
0.6371661200000072
New Jersey Devils vs. Toronto Maple Leafs
2017-10-11
2017020046
Time for simulation to run: 67.81296610832214
0.5246966499999978
Pittsburgh Penguins vs. Washington Capitals
2017-10-11
2017020047
Time for simulation to run: 66.91569590568542
0.5305323500000051
Ottawa Senators vs. Vancouver Canucks
2017-10-11
2017020044
Time for simulation to run: 69.90654802322388
0.4483713600000082
Boston Bruins vs. Colorado Avalanche
2017-10-12
2017020048
Time for simulation to run: 73.34377217292786
0.4237939899999997
New York Islanders vs. Anaheim Ducks
2017-10-12
2017020049
Time for simulation to run: 69.82223176956177
0.5126137600000013
Calgary Flames vs. Los Angeles Kings
2017-10-12
2017020050
Time for simulation to run: 68.51168012619019
0.5241764100000013
Pittsburgh Penguins vs. Tampa Bay Lightning
2017-10-12
2017020051
Time for simulation to run: 67.8569028377533
0.4729069100000022
St. Louis Blues vs. Florida Panthers
2017-10-12


/Users/MattBarlowe/.virtualenvs/databasescripts/lib/python3.6/site-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in long_scalars
/Users/MattBarlowe/.virtualenvs/databasescripts/lib/python3.6/site-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in long_scalars
/Users/MattBarlowe/.virtualenvs/databasescripts/lib/python3.6/site-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in long_scalars
/Users/MattBarlowe/.virtualenvs/databasescripts/lib/python3.6/site-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in long_scalars


...

Time for simulation to run: 66.37618088722229
0.47440185999999823
Tampa Bay Lightning vs. Edmonton Oilers
2018-02-06
2017020811
Time for simulation to run: 66.9049699306488
0.45962394000000106
Anaheim Ducks vs. Buffalo Sabres
2018-02-07
2017020812
Time for simulation to run: 67.31337380409241
0.42560534999999783
Vegas Golden Knights vs. Pittsburgh Penguins
2018-02-07
2017020813
Time for simulation to run: 64.75706171989441
0.48710888999998886
Philadelphia Flyers vs. Carolina Hurricanes
2018-02-07
2017020814
Time for simulation to run: 67.72295808792114
0.4676343300000012
Washington Capitals vs. Columbus Blue Jackets
2018-02-07
2017020815
Time for simulation to run: 67.49174404144287
0.4783739699999999
New Jersey Devils vs. Ottawa Senators
2018-02-07
2017020816
Time for simulation to run: 68.14491701126099
0.5205185700000079
Boston Bruins vs. Detroit Red Wings
2018-02-07
2017020817
Time for simulation to run: 67.07077670097351
0.43724090000000543
Vancouver Canucks vs. Florida Panthers
2

Columbus Blue Jackets vs. Toronto Maple Leafs
2018-02-15
2017020876
Time for simulation to run: 67.57287979125977
0.4817224799999928
Montréal Canadiens vs. Colorado Avalanche
2018-02-15
2017020877
Time for simulation to run: 67.4794340133667
0.45612572999999756
Florida Panthers vs. Vancouver Canucks
2018-02-15
2017020878
Time for simulation to run: 67.591068983078
0.45740532000000395
Carolina Hurricanes vs. New Jersey Devils
2018-02-16
2017020879
Time for simulation to run: 67.8580048084259
0.48868168999999323
New York Rangers vs. New York Islanders
2018-02-16
2017020880
Time for simulation to run: 66.35145425796509
0.46959019999999785
Los Angeles Kings vs. Pittsburgh Penguins
2018-02-16
2017020881
Time for simulation to run: 67.88931083679199
0.5371181899999921
Buffalo Sabres vs. Ottawa Senators
2018-02-16
2017020882
Time for simulation to run: 67.83634424209595
0.5315547499999987
Detroit Red Wings vs. Tampa Bay Lightning
2018-02-16
2017020883
Time for simulation to run: 67.3444778919

Minnesota Wild vs. New York Rangers
2018-02-24
2017020941
Time for simulation to run: 77.87687611579895
0.47360619999999826
Pittsburgh Penguins vs. Carolina Hurricanes
2018-02-24
2017020942
Time for simulation to run: 81.65427708625793
0.41198292000000236
Winnipeg Jets vs. St. Louis Blues
2018-02-24
2017020943
Time for simulation to run: 81.62225675582886
0.523643610000003
San Jose Sharks vs. Chicago Blackhawks
2018-02-24
2017020944
Time for simulation to run: 82.52573895454407
0.48861610999998967
Vancouver Canucks vs. Vegas Golden Knights
2018-02-24
2017020945
Time for simulation to run: 76.48222589492798
0.6282781700000023
Philadelphia Flyers vs. Ottawa Senators
2018-02-24
2017020946
Time for simulation to run: 74.52259683609009
0.4728071799999992
Colorado Avalanche vs. Calgary Flames
2018-02-24
2017020947
Time for simulation to run: 73.97022795677185
0.5569420599999924
New York Islanders vs. New Jersey Devils
2018-02-24
2017020952
Time for simulation to run: 76.81502175331116
0.4713

Ottawa Senators vs. Arizona Coyotes
2018-03-04
2017021006
Time for simulation to run: 67.93182110786438
0.4639910699999975
New York Rangers vs. Edmonton Oilers
2018-03-04
2017021007
Time for simulation to run: 67.85340118408203
0.49402022999999917
Philadelphia Flyers vs. Florida Panthers
2018-03-04
2017021008
Time for simulation to run: 67.63052797317505
0.48888713999999345
Nashville Predators vs. Colorado Avalanche
2018-03-04
2017021009
Time for simulation to run: 67.24082708358765
0.4211940199999986
Chicago Blackhawks vs. Anaheim Ducks
2018-03-04
2017021010
Time for simulation to run: 67.8644700050354
0.49403621999999653
Vegas Golden Knights vs. New Jersey Devils
2018-03-04
2017021011
Time for simulation to run: 64.90172481536865
0.4013025099999965
Winnipeg Jets vs. Carolina Hurricanes
2018-03-05
2017021012
Time for simulation to run: 66.84327912330627
0.44930856000000613
Detroit Red Wings vs. Minnesota Wild
2018-03-05
2017021013
Time for simulation to run: 66.79068183898926
0.568508

Time for simulation to run: 67.66031289100647
0.4264932199999993
Dallas Stars vs. Montréal Canadiens
2018-03-13
2017021072
Time for simulation to run: 68.05212640762329
0.49418697999999206
Ottawa Senators vs. Tampa Bay Lightning
2018-03-13
2017021073
Time for simulation to run: 67.27249670028687
0.564149350000002
Winnipeg Jets vs. Nashville Predators
2018-03-14
2017021074
Time for simulation to run: 66.82856869697571
0.5145147800000085
Colorado Avalanche vs. Minnesota Wild
2018-03-14
2017021075
Time for simulation to run: 66.24865102767944
0.5873593000000082
Edmonton Oilers vs. Calgary Flames
2018-03-14
2017021076
Time for simulation to run: 67.21033692359924
0.49376631999999265
Los Angeles Kings vs. Arizona Coyotes
2018-03-14
2017021077
Time for simulation to run: 68.35293197631836
0.41947465000000295
Dallas Stars vs. Toronto Maple Leafs
2018-03-14
2017021078
Time for simulation to run: 66.69571781158447
0.5217731700000047
Pittsburgh Penguins vs. New York Rangers
2018-03-15
2017021079